## Techniche - Topic model

In [50]:
import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs)# TODO (Lee) resolve

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
np.random.seed(3)

### Acquire data

In [70]:
%%capture
# acquire dataset of ML patents by making api call from PatentsView API 
data = get_ml_patents()

### Structure data

In [4]:
# specify fields (key:val pairs) to retain from full set of returned fields from api call
retained_keys = ['patent_number', 'patent_date', 'patent_title', 'patent_abstract']
data = trim_data(data=data, keys=retained_keys)

# create new key:value pair from combined values of patent_title and patent_abstract keys
data = create_title_abstract_col(data=data)

# create dataframe, organize columns and sort by patent_date
df = structure_dataframe(data=data)

#### Partition data

# partition data
data_train, data_test = partition_dataframe(df, .8)

# convert dataframe to list
text_data = df.patent_title_abstract.tolist()
text_train = data_train.patent_title_abstract.tolist()
text_train = data_test.patent_title_abstract.tolist()

### Pre-process text data

### Pre-process text data

In [56]:
# construct pipeline
# uncomment to download stop words
# !python -m spacy download en
stop_words = stopwords.words('/Users/lee/Documents/techniche/techniche/data/stopwords/english')

nlp = build_pipeline()
print(nlp.pipe_names)

# pre-process documents
processed_docs = process_docs(text_train)

### Build corpus and dictionary

In [66]:
# build dictionary
id_to_word = corpora.Dictionary(processed_docs)

# build corpus
texts = processed_docs

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(doc) for doc in processed_docs]

In [68]:
# view formatted corpus (term-doc-frequency)
# [[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

### Train model - model #1

In [71]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [76]:
# keywords in n topics in corpus
# pprint(model_lda.print_topics())

In [77]:
# most important keywords, and the respective weight, that form topic with index 0
# pprint(model_lda.print_topic(24))

### Evaluate - model #1

In [81]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
print(perplexity)

-5.409331488170752


/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [83]:
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=processed_docs, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
print(coherence_1)

0.8246017876742046


In [84]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

[0.9357145773844756,
 0.9994613403704132,
 0.9138137295660194,
 0.297304697943577,
 0.8764059594314315,
 0.9994613403704132,
 0.8242014874534215,
 0.957409697371036,
 0.9952506968788706,
 0.9661640808841557,
 0.8702955201314587,
 0.9994613403704132,
 0.9084723551468945,
 0.7210634511922711,
 0.8050818306878359,
 0.33760164940811865,
 0.28648942965597374,
 0.35312834226704537,
 0.9994613403704132,
 0.8129572343683117,
 0.9994613403704132,
 0.9994613403704132,
 0.949781565047193,
 0.9941831104462331,
 0.8129572343683116]

In [85]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.224365  0.008295       1        1  19.980911
4     -0.137569  0.054356       2        1  12.342148
7      0.092745 -0.248328       3        1  11.225811
12    -0.004663  0.160878       4        1  11.004402
2     -0.184915  0.080390       5        1  10.783062
6     -0.089538 -0.182812       6        1  10.556946
10    -0.069518 -0.122808       7        1   9.328748
8      0.269243  0.091253       8        1   8.661732
14    -0.118603  0.041202       9        1   5.762587
13     0.022359  0.009752      10        1   0.025664
17     0.032978 -0.000070      11        1   0.023602
16     0.028123  0.002724      12        1   0.023554
15     0.026406  0.007912      13        1   0.023486
9      0.030018  0.012946      14        1   0.022233
0      0.027452  0.007791      15        1   0.022204
24     0.025019  0.008006      16        1   0.022203
23     0.032173  0.008316      17        1   0.022165
19     0.028497  0.007728      18        1   0.022154
22     0.030951  0.006083      19        1   0.022142
18     0.030535  0.007731      20        1   0.020966
11     0.030535  0.007731      21        1   0.020656
5      0.030535  0.007731      22        1   0.020656
20     0.030535  0.007731      23        1   0.020656
21     0.030535  0.007731      24        1   0.020656
1      0.030535  0.007731      25        1   0.020656, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
25    Default  13.000000       response  13.000000  30.0000  30.0000
20    Default  29.000000        natural  29.000000  29.0000  29.0000
17    Default  32.000000       language  32.000000  28.0000  28.0000
43    Default   9.000000         device   9.000000  27.0000  27.0000
164   Default   9.000000       endpoint   9.000000  26.0000  26.0000
18    Default  21.000000            may  21.000000  25.0000  25.0000
205   Default   9.000000        metrics   9.000000  24.0000  24.0000
11    Default   9.000000         follow   9.000000  23.0000  23.0000
213   Default   8.000000        service   8.000000  22.0000  22.0000
69    Default   8.000000            cad   8.000000  21.0000  21.0000
134   Default   8.000000         entity   8.000000  20.0000  20.0000
29    Default  10.000000         system  10.000000  19.0000  19.0000
191   Default   6.000000       compound   6.000000  18.0000  18.0000
82    Default   9.000000        include   9.000000  17.0000  17.0000
15    Default   6.000000          input   6.000000  16.0000  16.0000
30    Default  10.000000           user  10.000000  15.0000  15.0000
174   Default   5.000000           item   5.000000  14.0000  14.0000
176   Default   5.000000     linguistic   5.000000  13.0000  13.0000
184   Default   5.000000      subsystem   5.000000  12.0000  12.0000
14    Default   6.000000    information   6.000000  11.0000  11.0000
233   Default   6.000000       messages   6.000000  10.0000  10.0000
190   Default   6.000000  communication   6.000000   9.0000   9.0000
3     Default  10.000000          based  10.000000   8.0000   8.0000
272   Default   5.000000       segments   5.000000   7.0000   7.0000
262   Default   5.000000        message   5.000000   6.0000   6.0000
162   Default   4.000000      dependent   4.000000   5.0000   5.0000
33    Default   6.000000        command   6.000000   4.0000   4.0000
124   Default   6.000000           data   6.000000   3.0000   3.0000
54    Default   6.000000            one   6.000000   2.0000   2.0000
2     Default   6.000000     associated   6.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
313   Topic25   0.000526    dehydration   1.166180   0.7817  -5.8636
351   Topic25   0.000526         volume   1.166178   0.7817  -5.8636
111   Topic25   0.000526      suppliers   1.207559   0.7468  -5.8636
112   Topic25  

### Model 2-  Mallet model

In [92]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update path
path_mallet = '/Users/lee/Documents/techniche/techniche/data/mallet-2.0.8/bin/mallet'

In [93]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [94]:
# topics
pprint(model_2.show_topics(formatted=False))

[(20,
  [('metrics', 0.25),
   ('service', 0.225),
   ('compound', 0.175),
   ('customer', 0.125),
   ('defined', 0.05),
   ('final', 0.025),
   ('correlations', 0.025),
   ('maintaining', 0.025),
   ('internet', 0.025),
   ('plurality', 0.025)]),
 (5,
  [('natural', 0.39622641509433965),
   ('command', 0.1320754716981132),
   ('module', 0.11320754716981132),
   ('control', 0.07547169811320754),
   ('language', 0.03773584905660377),
   ('parser', 0.03773584905660377),
   ('design', 0.03773584905660377),
   ('related', 0.03773584905660377),
   ('facilitate', 0.018867924528301886),
   ('topic', 0.018867924528301886)]),
 (16,
  [('processing', 0.21428571428571427),
   ('online', 0.10714285714285714),
   ('unambiguously', 0.10714285714285714),
   ('output', 0.07142857142857142),
   ('slots', 0.07142857142857142),
   ('relationships', 0.03571428571428571),
   ('inference', 0.03571428571428571),
   ('facilitating', 0.03571428571428571),
   ('understandability', 0.03571428571428571),
   ('dat

In [98]:
# TODO (Lee) - calculate coherence metric
coherence_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_to_word, coherence='c_v')
coherence_2 = coherence_2.get_coherence()
print(coherence_2)

/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:193: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:194: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:189: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs


nan


In [ ]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_to_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

### Model 3 - Author topic model

In [ ]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.head(3)

In [ ]:
df.head(3)

In [ ]:
# TODO (Lee) - resolve workaround
df_idx = df
df_idx['idx'] = df.index
df_idx
df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
df_idx_2 = df_idx_1.set_index('patent_number')
df_idx_2.pop('inventors')
df_idx_2
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_pat_idx = df_idx_2.T.to_dict('records')
for i in df_pat_idx:
    df_pat_idx = dict(i)
df_pat_idx

In [ ]:
df_inv_test = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inv_test.head(3)

In [ ]:
df_idx_pat_inv_map = df[['patent_number', 'inventors']]
df_idx_pat_inv_map.head(3)

In [ ]:
# TODO (Lee) - find out how to get list of patents_view_field names from API - I did it accidentally but need to replicate

In [ ]:
df.patent_title_abstract[0]

In [ ]:
df[:3]

In [ ]:
df_inventors.set_index('inventor_id').T.to_dict('list')

In [ ]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

In [ ]:
patdf2inv = dict((df_pat_idx[key], value) for (key, value) in pat2inv.items())
patdf2inv

In [ ]:
pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
pat2inv

In [ ]:
idx_pat_map = df.patent_number.to_dict()
idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
idx_pat_map

#### Construct author-topic model

In [ ]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=patdf2inv,
                         id2word=id_to_word, 
                         num_topics=25)

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [ ]:
# retrieve the topic distribution for an author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, but only the ones above a certain threshold are shown.

model_at['7788103-1']

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
from gensim.models import atmodel
doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs.

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

In [ ]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)